In [3]:
"""
Example using Pandas with the Iris dataset.
"""

import pandas as pd
from suricata import generate_report
from IPython.display import HTML

IRIS_URL = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv"

In [4]:
# Load Iris dataset directly using Pandas.
iris_df = pd.read_csv(IRIS_URL)
html_report = generate_report(iris_df, output_file="iris_report.html")
print("Pandas report saved as 'iris_report.html'")
HTML(html_report)

INFO:suricata.logger:summary_statistics executed in 0.0060 seconds
INFO:suricata.logger:missing_values executed in 0.0010 seconds
INFO:suricata.logger:correlation_matrix executed in 0.0000 seconds


Pandas report saved as 'iris_report.html'
